# 1. Loading im.db as pd.df

In [1]:
import pandas as pd
import sqlite3

In [2]:
import numpy as np
from scipy.stats import linregress

In [3]:
conn = sqlite3.connect('data/im.db')
cursor = conn.cursor()

In [4]:
imdb_basics = pd.read_sql('''
SELECT *
FROM movie_basics
''', conn)

imdb_akas = pd.read_sql('''
SELECT *
FROM movie_akas
''', conn)

imdb_ratings = pd.read_sql('''
SELECT *
FROM movie_ratings
''', conn)

imdb_principals = pd.read_sql('''
SELECT *
FROM principals
''', conn)

known_known_for = pd.read_sql('''
SELECT *
FROM known_for
''', conn)

imdb_directors = pd.read_sql('''
SELECT *
FROM directors
''', conn)

imdb_writers = pd.read_sql('''
SELECT *
FROM writers
''', conn)

imdb_persons = pd.read_sql('''
SELECT *
FROM persons
''', conn)

# 2. Name of df Variables:

### imdb Variables (each variable is an individual table):
- imdb_basics
- imdb_akas
- imdb_ratings
- imdb_principals
- known_known_for
- imdb_directors
- imdb_writers
- imdb_persons

# 3. Exploring IMDB df's:

### 3a. Exploring: movie_akas_imdb

In [ ]:
#movie_akas_imdb.info()

#### ordering = 1 has the same length as indivitual movie_id's be the same length as unique movie_id

In [5]:
imdb_akas = pd.read_sql('''
SELECT *
FROM movie_akas
WHERE region = 'US'
GROUP BY movie_id
''', conn)
imdb_akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47433 entries, 0 to 47432
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie_id           47433 non-null  object 
 1   ordering           47433 non-null  int64  
 2   title              47433 non-null  object 
 3   region             47433 non-null  object 
 4   language           325 non-null    object 
 5   types              4977 non-null   object 
 6   attributes         1652 non-null   object 
 7   is_original_title  47428 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 2.9+ MB


In [ ]:
#imdb_akas['movie_id'].value_counts()

In [ ]:
len(imdb_akas['movie_id'].unique())

In [ ]:
#imdb_akas.loc[m_akas_us['movie_id'] == 'tt2488496']

### 3b. Exploring: movie_akas_imdb

In [ ]:
#movie_basics_imdb.info()

In [ ]:
#movie_basics_imdb['movie_id'].value_counts()

In [ ]:
akas_basics_merge = imdb_akas.merge(imdb_basics, on='movie_id',
                                        how='inner')
imdb_basics.info()

In [ ]:
akas_basics_merge['movie_id'].value_counts()

### 3c. Explored: movie_rat_imdb

In [ ]:
#movie_rat_imdb.info()

 # 3d. Combining IMDB database

In [ ]:
imdb_merged = akas_basics_merge.merge(imdb_ratings, on='movie_id',
                                       how='left')
imdb_merged

In [ ]:
#imdb_merged.info()

In [ ]:
#imdb_merged.drop(['language', 'attributes', 'types'], axis=1, inplace=True)

In [ ]:
imdb_merged.info()

# 4. Loading tn.movie_budgets as pd.df

In [ ]:
tn_budgets = pd.read_csv('data/tn.movie_budgets.csv.gz')

# 5. Exploring tn_budgets

In [ ]:
tn_budgets.drop_duplicates(subset = ['movie'], inplace=True)

In [ ]:
tn_budgets.rename(columns = {'movie':'primary_title'}, inplace=True)

In [ ]:
tn_budgets.info()

In [ ]:
#Creating new coumn called year that is date.time
tn_budgets['Release_Date_Time'] = pd.to_datetime(tn_budgets['release_date'])
tn_budgets['start_year'] = tn_budgets['Release_Date_Time'].dt.year

# 6: MERGE Merging Budgets to IMDB

#### follow link: https://www.geeksforgeeks.org/how-to-convert-pandas-dataframe-into-sql-in-python/

In [ ]:
imdb_db = imdb_merged.merge(tn_budgets, on= ['primary_title','start_year'], how= 'inner')

In [ ]:
imdb_db['worldwide_gross'] = imdb_db['worldwide_gross'].str.replace('$','').str.replace(',','').astype(float)
imdb_db['production_budget'] = imdb_db['production_budget'].str.replace('$','').str.replace(',','').astype(float)
imdb_db['domestic_gross'] = imdb_db['domestic_gross'].str.replace('$','').str.replace(',','').astype(float)

# Dropping Tables
- get rid of start_year or release_date
-

# Analysis:
# Sorted by Genre

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# same as 
# from matplotlib import pyplot as plt

import seaborn as sns

from scipy import stats
from sklearn.datasets import load_iris

In [ ]:
imdb_db['genres'].dropna(inplace=True)

In [ ]:
imdb_db['worldwide_gross'].dropna(inplace=True)

In [ ]:
genre_profit = imdb_db.loc[:,['genres','worldwide_gross']]

In [ ]:
genre_profit.dropna(axis = 0, inplace=True)

In [ ]:
gen_pro = genre_profit.sort_values('worldwide_gross', ascending=False)

In [ ]:
#gen_pro

In [ ]:
imdb_db.info()

In [ ]:
gen_pro.plot(kind = 'bar',
        x = 'genres',
        y = 'profit',
        color = 'blue')
  
# set the title
plt.title('Top Performing Genres')
  
# show the plot
plt.show()

In [ ]:
gen_pro.plot(kind = 'scatter',
        x = 'genres',
        y = 'worldwide_gross',
        color = 'blue')
  
# set the title
plt.title('Top Performing Genres')
  
# show the plot
plt.show()

In [ ]:
gen_pro.plot(kind = 'hist',
        x = 'genres',
        y = 'worldwide_gross',
        color = 'blue')
  
# set the title
plt.title('Top Performing Genres')
  
# show the plot
plt.show()

In [ ]:
correlation_matrix = imdb_db.corr()
print(correlation_matrix)

# EXTRA CREDIT STUFF (ORDERING BY CAST)

In [ ]:
#movie_akas_imdb.loc[movie_akas_imdb['ordering'] == 1]

In [ ]:
ordering_df = movie_akas_imdb.loc[movie_akas_imdb['ordering'] == 1]

In [ ]:
#ordering_df.info()

In [ ]:
#principals_imdb['category']

In [ ]:
principals_imdb = pd.read_sql(''' SELECT * FROM principals ''', conn) principals_imdb.loc[principals_imdb['movie_id'] == 'tt0323808']

In [ ]:
# Ordering = IMDB Top Cast order

I figured it out! It's the order that the 'top cast' shows up on the IMDB page. 

I'm going to stash this for after I make a visualization for Genre. 
Going to see if the 'top cast' 

search google for id = 'tt0323808'

https://www.imdb.com/title/tt0323808/fullcredits?ref_=tt_cl_sm
    
principals_imdb = pd.read_sql('''
SELECT *
FROM principals
''', conn)
principals_imdb.loc[principals_imdb['movie_id'] == 'tt0323808']

In [ ]:
imdb_order =pd.read_sql('''
SELECT *, sum(ordering)
FROM principals
WHERE ordering <= 
GROUP BY movie_id
ORDER BY sum(ordering) desc
''', conn)
imdb_order

In [ ]:
#principals_imdb.loc[principals_imdb['ordering'] == 10]

In [ ]:
#principals_imdb.loc[principals_imdb['movie_id'] == 'tt0323808']

In [ ]:
#principals_imdb['movie_id'].value_counts()